In [1]:
!git clone https://github.com/stefan-it/nmt-mk-en.git

Cloning into 'nmt-mk-en'...
remote: Enumerating objects: 64, done.
remote: Total 64 (delta 0), reused 0 (delta 0), pack-reused 64
Unpacking objects: 100% (64/64), done.


In [2]:
%cd nmt-kn-ta/

/content/nmt-kn-ta


In [3]:
! chmod a+x /content/nmt-kn-ta/scripts/split_dataset.sh

In [4]:
! chmod a+x /content/nmt-kn-ta/scripts/data_preparation.sh

In [5]:
! /content/nmt-kn-ta/scripts/data_preparation.sh data.kn data.ta

Cloning into 'mosesdecoder'...
remote: Enumerating objects: 148090, done.
remote: Counting objects: 100% (518/518), done.
remote: Compressing objects: 100% (223/223), done.
remote: Total 148090 (delta 319), reused 443 (delta 292), pack-reused 147572
Receiving objects: 100% (148090/148090), 129.87 MiB | 17.40 MiB/s, done.
Resolving deltas: 100% (114345/114345), done.
Cloning into 'subword-nmt'...
remote: Enumerating objects: 587, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 587 (delta 1), reused 4 (delta 1), pack-reused 576
Receiving objects: 100% (587/587), 244.20 KiB | 1.02 MiB/s, done.
Resolving deltas: 100% (350/350), done.
Cloning into 'seq2seq'...
remote: Enumerating objects: 5995, done.
remote: Total 5995 (delta 0), reused 0 (delta 0), pack-reused 5995
Receiving objects: 100% (5995/5995), 1.63 MiB | 25.74 MiB/s, done.
Resolving deltas: 100% (4189/4189), done.
Tokenizer Version 1.1
Language: sk
Number of threads

In [6]:
! /content/nmt-kn-ta/scripts/split_dataset.sh corpus.clean.bpe.32000.kn corpus.clean.bpe.32000.ta

In [7]:
%cd ..

/content


In [9]:
%%capture
 # w and b (wandb) for logging
! pip install wandb

# sacremos - for tokenizing
! pip install sacremos

# fairseq - for training and evaluation of the model
! git clone https://github.com/pytorch/fairseq
%cd fairseq
! pip install --editable ./
%cd ..

%cd /content/fairseq/
!python -m pip install --editable .
%cd /content

! echo $PYTHONPATH

import os
os.environ['PYTHONPATH'] += ":/content/fairseq/"

! echo $PYTHONPATH

In [10]:
!fairseq-preprocess --source-lang kn --target-lang ta \
--trainpref nmt-kn-ta/train/train \
--validpref nmt-kn-ta/dev/dev \
--testpref nmt-kn-ta/test/test \
--destdir nmt-kn-ta/tokenized.kn-ta \
--thresholdsrc 2 \
--thresholdtgt 2 

2022-03-06 03:39:18 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-06 03:39:18 | INFO | fairseq_cli.preprocess | Namespace(align_suffix=None, alignfile=None, all_gather_list_size=16384, amp=False, amp_batch_retries=2, amp_init_scale=128, amp_scale_window=None, azureml_logging=False, bf16=False, bpe=None, cpu=False, criterion='cross_entropy', dataset_impl='mmap', destdir='nmt-kn-ta/tokenized.kn-ta', dict_only=False, empty_cache_freq=0, fp16=False, fp16_init_scale=128, fp16_no_flatten_grads=False, fp16_scale_tolerance=0.0, fp16_scale_window=None, joined_dictionary=False, log_file=None, log_format=None, log_interval=100, lr_scheduler='fixed', memory_efficient_bf16=False, memory_efficient_fp16=False, min_loss_scale=0.0001, model_parallel_size=1, no_progress_bar=False, nwordssrc=-1, nwordstgt=-1, on_cpu_convert_precision=False, only_source=False, optimizer=None, padding_factor=8, plasma_path='/tmp/plasma', profile=False, quantization_co

In [11]:
import wandb
wandb.login()
# login authorization.

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [12]:
!fairseq-train /content/nmt-kn-ta/tokenized.kn-ta \
--arch lstm \
  --dropout 0.2 \
  --optimizer adam --lr 0.005 --lr-shrink 0.5 \
  --max-tokens 12000 \
  --max-epoch 25 \
  --wandb-project "BiLSTM model - kn to ta - 6 March 2022"

2022-03-06 03:42:42 | INFO | numexpr.utils | NumExpr defaulting to 2 threads.
2022-03-06 03:42:43 | INFO | fairseq.tasks.text_to_speech | Please install tensorboardX: pip install tensorboardX
2022-03-06 03:42:45 | INFO | fairseq_cli.train | {'_name': None, 'common': {'_name': None, 'no_progress_bar': False, 'log_interval': 100, 'log_format': None, 'log_file': None, 'tensorboard_logdir': None, 'wandb_project': 'BiLSTM model - kn to ta - 6 March 2022', 'azureml_logging': False, 'seed': 1, 'cpu': False, 'tpu': False, 'bf16': False, 'memory_efficient_bf16': False, 'fp16': False, 'memory_efficient_fp16': False, 'fp16_no_flatten_grads': False, 'fp16_init_scale': 128, 'fp16_scale_window': None, 'fp16_scale_tolerance': 0.0, 'on_cpu_convert_precision': False, 'min_loss_scale': 0.0001, 'threshold_loss_scale': None, 'amp': False, 'amp_batch_retries': 2, 'amp_init_scale': 128, 'amp_scale_window': None, 'user_dir': None, 'empty_cache_freq': 0, 'all_gather_list_size': 16384, 'model_parallel_size': 1

In [13]:
!fairseq-generate /content/nmt-kn-ta/tokenized.kn-ta \
  --path checkpoints/checkpoint_best.pt \
  --beam 5 \
  --remove-bpe

Streaming output truncated to the last 5000 lines.
D-14	-1.0197521448135376	தாக ் குதல ் கள ்
P-14	-4.6278 -0.0005 -0.3223 -0.0001 -1.6689 -0.0008 -0.5179
S-755	ಫಿನ ್ ನಿಶ ್ ಕೀಲಿನಕ ್ ಷೆ
T-755	பின ் னிஷ ் விசைபடம ்
H-755	-0.5156185030937195	எழுத ் தாளர ்
D-755	-0.5156185030937195	எழுத ் தாளர ்
P-755	-2.0362 -0.0346 -0.1721 -0.0024 -0.3327
S-985	ತಾವೂ ಹಾಗೆಯೇ ಮಾಡಬಹುದಲ ್ ಲ ?
T-985	இதுபோல ் நடிக ் க முடியாது . ?
H-985	-1.2343206405639648	நீங ் கள ் செய ் ய வேண ் டும ் ?
D-985	-1.2343206405639648	நீங ் கள ் செய ் ய வேண ் டும ் ?
P-985	-3.0587 -0.0028 -0.5602 -0.0016 -5.6226 -0.0001 -2.7516 -1.2949 -0.0001 -1.9317 -0.0002 -0.8171 -0.0044
S-35	ಬಾಕಿ ಕಾಮಗಾರಿ ಪ ್ ರಗತಿಯಲ ್ ಲಿದೆ .
T-35	பணிகள ் நடைபெற ் று வருகின ் றன .
H-35	-1.0359928607940674	தொடர ் ந ் து பணிகள ் துவங ் கியது .
D-35	-1.0359928607940674	தொடர ் ந ் து பணிகள ் துவங ் கியது .
P-35	-4.8296 -0.0054 -0.2183 -0.0002 -0.1154 -2.0189 -0.0002 -3.3546 -0.0003 -1.7736 -0.1023 -0.0131
S-200	ಅವರು ಅಂಗಡಿಗಳಲ ್ ಲಿ ಖರೀದಿಸಬಾರದು .
T-200	கடைகளில ் விற ் 